# Trajectory Optimization for a planar walker

## Start meshcat visualizer

Open the visualizer by clicking on the url in the output of this cell.

In [1]:
from pydrake.all import FindResourceOrThrow, StartMeshcat, MeshcatVisualizer
import matplotlib.pyplot as plt
import numpy as np
import time
from pydrake.math import RigidTransform
from pydrake.all import (
    DiagramBuilder, Simulator, FindResourceOrThrow, MultibodyPlant, PiecewisePolynomial, SceneGraph,
    Parser, MultibodyPositionToGeometryPose, TrajectorySource, Demultiplexer, ConstantVectorSource
)
import importlib
import find_throwing_trajectory
importlib.reload(find_throwing_trajectory)
from direct_col_dev import find_throwing_trajectory

meshcat = StartMeshcat()


INFO:drake:Meshcat listening for connections at http://localhost:7002


## Run trajectory optimization to find the optimal trajectory

In [2]:
N = 10
initial_state = np.zeros(14)
q = np.zeros((7,))
q[0] = 0; q[1] = 0.8
theta = -np.arccos(q[1])
q[3] = theta/2; q[4] = -2 * theta
q[5] = theta;   q[6] = -2 * theta/2
initial_state[:7] = q
final_state = initial_state
# final_configuration = np.array([np.pi, 0])
tf = 3.0
x_traj, u_traj, prog,  _, _ = find_throwing_trajectory(N, initial_state, 1.3, tf=tf, jumpheight_tol=5e-2)


Starting the solve
optimal cost:  209290.2260216364
x_sol:  [[  0.     0.8   -0.    -0.32   1.29  -0.64   0.64   0.     0.     0.
    0.     0.     0.     0.  ]
 [  0.54   1.32   0.96  -0.77   3.56   1.2    0.66   2.83   2.62   5.37
    0.96  -1.19  -1.23  11.17]
 [  1.63   1.72   1.76  -2.15   4.56   0.53   1.97   3.51  -0.92  -4.19
  -16.22  -2.   -14.31  -9.48]
 [  2.3    1.8   -2.31 -10.84   5.15  -8.8    0.31   0.27   1.9   -1.51
  -14.39  31.31 -15.67  21.04]
 [  2.72   3.2    3.55  -3.21   5.42   1.17   6.39   3.85   4.59  17.23
   31.18 -26.23  53.15 -51.29]
 [  3.74   4.87   7.04  13.58   0.69   1.27   1.01   0.41   6.68   0.97
   26.69 -26.66 -11.89  41.66]
 [  4.83   7.31   6.92   1.53   2.28   8.92  -2.42   3.5    6.93   2.95
  -59.5   86.73  18.08  15.65]
 [  5.76   9.77   5.67   0.59   1.15  -2.24   3.55   4.49  10.05   0.34
   27.11 -29.18 -22.08 -24.66]
 [  7.14  14.     4.38   4.66  -4.75  13.36  -1.26   3.97  12.79  -2.3
   45.38 -76.48  -5.76 -31.44]
 [  8.16  17.53 

## Visualize the optimal trajectory

This will animate the optimal trajectory in the meshcat window (see above)

In [3]:
from pydrake.all import AddMultibodyPlantSceneGraph, HalfSpace, CoulombFriction, MeshcatVisualizerParams
# Create a MultibodyPlant for the arm
file_name = "/home/anirudhkailaje/Documents/01_UPenn/02_MEAM5170/03_FinalProject/src/planar_walker.urdf"

builder = DiagramBuilder()

#### Designing our world ####
# Add a planar walker to the simulation
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0005)
#Half space means a plane -> Ground Plane in particular
X_WG = HalfSpace.MakePose(np.array([0,0, 1]), np.zeros(3,))
plant.RegisterCollisionGeometry(plant.world_body(), X_WG, HalfSpace(), 
    "collision", CoulombFriction(1.0, 1.0))

#Make the plant
parser = Parser(plant)
parser.AddModels(file_name)
plant.WeldFrames(plant.world_frame(),
    plant.GetBodyByName("base").body_frame(),
    RigidTransform.Identity())
plant.Finalize()



# builder = DiagramBuilder()
# scene_graph = builder.AddSystem(SceneGraph())
# robot = builder.AddSystem(MultibodyPlant(0.0))
# robot.RegisterAsSourceForSceneGraph(scene_graph)
# Parser(plant=robot).AddModels(file_name)
# robot.WeldFrames(
#             robot.world_frame(),
#             robot.GetBodyByName("base").body_frame(),
#             RigidTransform.Identity()
# )
# robot.Finalize()

n_q = plant.num_positions()
n_v = plant.num_velocities()
n_u = plant.num_actuators()
print(n_q, n_v, n_u)

# x_traj_mod = np.zeros((n))
x_traj_source = builder.AddSystem(TrajectorySource(x_traj))
u_traj_source = builder.AddSystem(TrajectorySource(u_traj))

demux = builder.AddSystem(Demultiplexer(np.array([n_q, n_v])))
to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
zero_inputs = builder.AddSystem(ConstantVectorSource(np.zeros(n_u)))

builder.Connect(u_traj_source.get_output_port(), plant.get_actuation_input_port())
# builder.Connect(x_traj_source.get_output_port(), demux.get_input_port())
# builder.Connect(demux.get_output_port(0), to_pose.get_input_port())
# builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

# MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

vis_params = MeshcatVisualizerParams(publish_period=0.0005)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, params=vis_params)



diagram = builder.Build()
diagram.set_name("diagram")

simulator = Simulator(diagram)
simulator.Initialize()
plant_context = diagram.GetMutableSubsystemContext(plant, simulator.get_mutable_context())

q = np.zeros((plant.num_positions(),))
q[0] = 0; q[1] = 0.8
theta = -np.arccos(q[1])
q[3] = theta/2; q[4] = -2 * theta
q[5] = theta;   q[6] = -2 * theta/2
plant.SetPositions(plant_context, q)



  
simulator.set_target_realtime_rate(1)
simulator.AdvanceTo(tf);
timesteps = np.linspace(0, tf, 100)
com_pos_list = []
vel_list = []
for t in timesteps:
  x = x_traj.vector_values([t])
  u = np.linalg.norm(u_traj.vector_values([t]))
  plant.SetPositionsAndVelocities(plant_context, x)
  com_pos = plant.CalcCenterOfMassPositionInWorld(plant_context).ravel()
  com_pos_list.append(com_pos[2])
  vel_list.append(u)

7 7 4


RuntimeError: MultibodyPlant's discrete update solver failed to converge at simulation time = 1.237 with discrete update period = 0.0005. This usually means that the plant's discrete update period is too large to resolve the system's dynamics for the given simulation conditions. This is often the case during abrupt collisions or during complex and fast changing contact configurations. Another common cause is the use of high gains in the simulation of closed loop systems. These might cause numerical instabilities given our discrete solver uses an explicit treatment of actuation inputs. Possible solutions include:
  1. reduce the discrete update period set at construction,
  2. decrease the high gains in your controller whenever possible,
  3. switch to a continuous model (discrete update period is zero),      though this might affect the simulation run time.

In [ ]:
fig = plt.figure()
plt.plot(timesteps, com_pos_list)
plt.plot(timesteps,vel_list)
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=71c969e2-d88b-4bfd-96cb-c4ea4f7f9122' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>